In [403]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from nltk.corpus import stopwords
from stemming.porter2 import stem
from random import shuffle
from re import sub
from numpy.random import rand
from numpy import shape, exp, log, array

In [29]:
neg_l = list("-1 %s" % line.rstrip() for line in open('../../data/rt-polaritydata/rt-polarity.neg'))
pos_l = list("+1 %s" % line.rstrip() for line in open('../../data/rt-polaritydata/rt-polarity.pos'))

sen_l = neg_l + pos_l
shuffle(sen_l)

print(len(list(filter(lambda l: l[0:2] == '+1', sen_l))))
print(len(list(filter(lambda l: l[0:2] == '-1', sen_l))))

5330
5331


In [57]:
stop_words = stopwords.words('english') + [',', '.']
check = lambda x: x not in stop_words

In [90]:
y = list(); features = list()

for line in sen_l:
    y.append(line[0:2])
    features.append(line[3:])
    
cv = CountVectorizer()
vocab_size = len(set(sum(map(tokenize, features), [])))

In [149]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = lambda text: list(map(stem, sub("[^a-zA-Z]", " ", text).split())),
    lowercase = True,
    stop_words = 'english',
    max_features = int(vocab_size * 0.5) # ユニークな単語数のうち6割を使う、int()で整数にしている
)

X = vectorizer.fit_transform(features).todense()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [518]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [519]:
model.score(X_test, y_test)

0.74367718101733449

In [559]:
class MyLogisticRegression:
    def __init__(self):
        self.threshold = 0.7
    
    
    def h(self, X):
        z = X.dot(self.w)
        return 1 / (1 + exp(-z))
        
    
    def fit(self, X, y):
        self.w = rand(X.shape[1], 1)
        for t in range(10):
            w_tmp = self.w
            self.w = w - 0.01 * X.T.dot(self.h(X) - y) # gradient
    
    
    def predict(self, X):
        return array([int(res) for res in self.h(X) > self.threshold]).reshape(X.shape[0], 1)
   

    def score(self, X, y):
        res = self.predict(X)
        num_correct = len(list(filter(lambda x: x==1, res == y))) 
        return num_correct / res.shape[0]

In [560]:
mymodel = MyLogisticRegression()
mymodel.fit(X_train, y_train_i)

In [561]:
mymodel.score(X_test, y_test_i)

0.6007388462631429

In [584]:
len(vectorizer.get_feature_names())

6154

In [586]:
mymodel.w.shape

(6154, 1)